In [23]:
import numpy as np 
import pandas as pd 
from tqdm.notebook import tqdm
import json
from bertopic import BERTopic
from umap import UMAP

# custom scripts
from text_preprocessing import text_preprocessing
%matplotlib inline

# Num of lines per json chunks:  
* num of lines in test5.json is 1354309
* num of lines in test4.json is 1000000
* num of lines in test3.json is 1000000
* num of lines in test1.json is 1000000
* num of lines in test2.json is 1000000


In [2]:
# path = 'data/articles_data/'
# chunks = os.listdir(path)
# all_keys = set()
# count = 0
# for elem in chunks:
#     with open(path + elem, 'rb') as f:
#         cnt = 0
#         for _ in range(10**7):
#             try:
#                 all_keys.update(json.loads(next(f)))
#                 cnt += 1
#             except StopIteration:
#                 print(f'num of lines in {elem} is {cnt}')
#                 break
# print(all_keys)

# Abstract preprocessing

In [3]:
# already saved proprocessed texts
abstracts = np.load('all_data/preprocessed_abstracts_100k.npy', allow_pickle = True) # took only first 100k papers

In [4]:
# preproc in the same way
# for idx, paper in enumerate(tqdm(abstracts)):
#     abstracts[idx][1] = text_preprocessing(paper[1])

In [5]:
# for reproducybility 
# https://maartengr.github.io/BERTopic/faq.html#why-are-the-results-not-consistent-between-runs
umap_model = UMAP(n_neighbors = 15, 
                  n_components = 5, 
                  min_dist = 0.0,
                  metric = 'cosine', 
                  random_state = 42)

model = BERTopic(language = "english",
                 umap_model = umap_model,
                 verbose = True)
topics, probs = model.fit_transform(abstracts[:, 1])
topics_info = model.get_topic_info();

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

2022-11-08 18:43:19,825 - BERTopic - Transformed documents to Embeddings
2022-11-08 18:45:04,120 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2022-11-08 18:45:10,199 - BERTopic - Clustered reduced embeddings


In [6]:
topics_info

,Topic,Count,Name
0,-1,48845,-1_graph_problem_design_application
1,0,2512,0_protein_gene_genome_molecular
2,1,1116,1_student_course_learning_teacher
3,2,858,2_equation_numerical_boundary_wave
4,3,784,3_game_auction_player_equilibrium
...,...,...,...
819,818,10,818_supplier_selection_fanp_pfigp
820,819,10,819_sharing_repository_sciport_collaboration
821,820,10,820_dvb_deinterleaver_dmb_broadcasting
822,821,10,821_reliability_atr_gn_upperbound


In [7]:
model.save('bert_model_100k')

/opt/conda/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


# Example of the model inference

In [12]:
bert_model = BERTopic.load('bert_model_100k')

In [13]:
bert_model.get_topic_info()

,Topic,Count,Name
0,-1,48845,-1_graph_problem_design_application
1,0,2512,0_protein_gene_genome_molecular
2,1,1116,1_student_course_learning_teacher
3,2,858,2_equation_numerical_boundary_wave
4,3,784,3_game_auction_player_equilibrium
...,...,...,...
819,818,10,818_supplier_selection_fanp_pfigp
820,819,10,819_sharing_repository_sciport_collaboration
821,820,10,820_dvb_deinterleaver_dmb_broadcasting
822,821,10,821_reliability_atr_gn_upperbound


In [14]:
# upload raw abstracts
test_data = np.load('all_data/abstracts.npy', allow_pickle = True)[10**6 : 10**6 + 20]
# preprocess and score on the go
preds = {}
for idx, paper in tqdm(enumerate(test_data)):
    preprocesed_abstract = text_preprocessing(paper[1])
    topics, prob = bert_model.transform(preprocesed_abstract)
    preds[paper[0]] = topics_info[topics_info['Topic'] == topics[0]]['Name'].iloc[0]
    
for key, value in preds.items():
    print(key, ' : ', value)

0it [00:00, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-11-08 18:47:51,073 - BERTopic - Reduced dimensionality
2022-11-08 18:47:51,075 - BERTopic - Predicted clusters


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-11-08 18:47:51,933 - BERTopic - Reduced dimensionality
2022-11-08 18:47:51,942 - BERTopic - Predicted clusters


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-11-08 18:48:05,236 - BERTopic - Reduced dimensionality
2022-11-08 18:48:05,238 - BERTopic - Predicted clusters


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2022-11-08 18:48:05,949 - BERTopic - Reduced dimensionality
2022-11-08 18:48:05,950 - BERTopic - Predicted clusters


53e9b06fb7602d9703ad307e  :  -1_graph_problem_design_application
53e9b06fb7602d9703ad3097  :  -1_graph_problem_design_application
53e9b06fb7602d9703ad30c6  :  143_ecg_fetal_heart_qrs
53e9b06fb7602d9703ad30dc  :  -1_graph_problem_design_application
53e9b06fb7602d9703ad3112  :  -1_graph_problem_design_application
53e9b06fb7602d9703ad3125  :  -1_graph_problem_design_application
53e9b06fb7602d9703ad3155  :  -1_graph_problem_design_application
53e9b06fb7602d9703ad31ac  :  -1_graph_problem_design_application
53e9b06fb7602d9703ad31b7  :  604_option_price_american_martingale
53e9b06fb7602d9703ad31e4  :  74_painting_sketch_stroke_paint
53e9b06fb7602d9703ad31f5  :  -1_graph_problem_design_application
53e9b06fb7602d9703ad320f  :  143_ecg_fetal_heart_qrs
53e9b06fb7602d9703ad322c  :  19_grid_resource_job_scheduling
53e9b06fb7602d9703ad326a  :  46_testing_test_suite_coverage
53e9b06fb7602d9703ad326f  :  -1_graph_problem_design_application
53e9b06fb7602d9703ad3280  :  -1_graph_problem_design_applicat

P.S. The first number in topics defines the class, so you can take it from there.

# Save scored ids and their labels

In [94]:
topics_dict = dict(zip(bert_model.get_topic_info()['Topic'], bert_model.get_topic_info()['Name']))
result = []
for idx, elem in enumerate(bert_model.topics_):
    result.append([abstracts[idx][0], topics_dict[elem]])
result = np.array(result)
np.save('all_data/scored_data_100k.npy', result)

# Finally save valid papers' ids

In [47]:
abst = np.load('all_data/abstracts.npy', allow_pickle = True)
scored_abst = np.load('all_data/preprocessed_abstracts_100k.npy', allow_pickle = True)

In [48]:
valid_id = abst[:, 0].tolist()

with open('all_data/valid_id.json', 'w') as f:
    f.write(json.dumps({'id': valid_id}))
    
data = {}
with open('all_data/valid_id.json', 'rb') as f:
    data = json.loads(next(f))

In [49]:
len(data['id'])

3010729

In [50]:
scores_id = scored_abst[:, 0].tolist()

with open('all_data/scored_id.json', 'w') as f:
    f.write(json.dumps({'id': scores_id}))
    
scored_data = {}
with open('all_data/scored_id.json', 'rb') as f:
    scored_data = json.loads(next(f))

In [51]:
len(scored_data['id'])

100000